<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Enterprise Feature Store - Feature Store
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial;'><b>Enterprise Feature Store</b></p>
<p style = 'font-size:16px;font-family:Arial;'>This notebook demonstrates how to build, manage, and utilize an enterprise feature store using TeradataML. It covers the end-to-end workflow for feature engineering, ingestion, cataloging, and governance of features derived from business data such as sales, marketing, and transactions. <br>
<b>Notebook Purpose:</b>
<ul>
  <li>Show how to create and manage a centralized feature repository for analytics and machine learning.</li>
  <li>Demonstrate feature engineering and ingestion from raw business datasets.</li>
  <li>Illustrate feature lineage, versioning, and governance for reproducible ML workflows.</li>
</ul>


<p style = 'font-size:18px;font-family:Arial;'><b>Disclaimer</b></p>

<p style = 'font-size:12px;font-family:Arial;'>
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>1. Connect to Vantage, Import python packages and explore the dataset</b></p>

In [ ]:
!pip install teradataml==20.0.0.7 --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please execute the above pip install to get the latest version of the required library. Be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
from teradataml import *
from getpass import getpass
from collections import OrderedDict
from teradatasqlalchemy import INTEGER, FLOAT, VARCHAR, DATE
from sqlalchemy import literal_column
import warnings
warnings.filterwarnings('ignore')

display.max_rows = 5

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'> 1.1 Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=EFS-FeatureStore.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>2. Setup a Feature Store Repository</b></p>
<p style = 'font-size:18px;font-family:Arial;'><b>2.1 Create the FeatureStore</b></p>

In [ ]:
# Create a repo. 
fs = FeatureStore(repo='enterprise_feature_repo')

<p style = 'font-size:18px;font-family:Arial;'><b>2.2 Setup the FeatureStore</b></p>

In [ ]:
fs.setup()

<p style = 'font-size:18px;font-family:Arial;'><b>2.3 List the available feature stores</b></p>

In [ ]:
FeatureStore.list_repos()

<p style = 'font-size:18px;font-family:Arial;'><b>2.4 Checking Availability</b></p>

In [ ]:
# First initiate feature store for the repo enterprise_feature_repo. Keep the data domain as analytics.
fs = FeatureStore(repo='enterprise_feature_repo', data_domain='Analytics')

<p style = 'font-size:18px;font-family:Arial;'><b>Feature Engineering</b></p>
<p style = 'font-size:16px;font-family:Arial;'>In this feature engineering process, we will use the <code>teradataml</code> package to create Teradata DataFrames that implement the required computations. The two main feature engineering tasks are:</p>
<ol>
    <li>
        <strong>Statistics on Customer:</strong> For each <code>customer ID</code>, we will compute:
        <ul>
            <li>The sum of all transaction amounts.</li>
            <li>The average amount per transaction.</li>
            <li>The total number of transactions.</li>
            <li>The number of days since the last transaction.</li>
        </ul>
    </li>
    <li>
        <strong>Spending Category Distribution:</strong> For each transaction category, we will compute:
        <ul>
            <li>The sum of transaction amounts.</li>
            <li>The mean, standard deviation, maximum, and median of transaction amounts.</li>
        </ul>
    </li>
</ol>

<p style = 'font-size:16px;font-family:Arial;'>These computations will result in two Teradata DataFrames:</p>

<ul>
    <li><code>df_eng_feat_cust</code>: Features computed per customer.</li>
    <li><code>df_eng_feat_cat</code>: Features computed for spending category distribution.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;'>Note that these DataFrames only implement the processing logic and do not generate data until explicitly stored or exported. When displaying the content of these DataFrames, only a sample of the results will be shown. To generate the actual data, you would need to either:</p>
<ul>
    <li>Store the results in another table within the database.</li>
    <li>Export the results to a <code>pandas</code> DataFrame, files, ...</li>
</ul>
</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>3. Get Data For demo</b>
<p style = 'font-size:18px;font-family:Arial;'><b>3.1 Load the transaction Data</b></p>

In [ ]:
t_types = OrderedDict(CustomerID=INTEGER, Transaction_Amount=FLOAT, Date_transaction=DATE, Category=VARCHAR(200), MerchantID=INTEGER)
df = read_csv(table_name='transactions',
               filepath=r"data/transactions.csv",
              types=t_types)

<p style = 'font-size:18px;font-family:Arial;'><b>3.2 Perform Data Transformation</b></p>

In [ ]:
df_eng_feat_cust = df.groupby('CustomerID').agg({'Transaction_Amount' : ['sum','mean','count'], 'Date_transaction':['max']})

<p style = 'font-size:16px;font-family:Arial;'><b>Statistics on customers </b></p>

In [ ]:
df_eng_feat_cust = df.groupby('CustomerID').assign(total_Transaction_Amount=df.Transaction_Amount.sum(),
                                                   avg_Transaction_Amount=df.Transaction_Amount.mean(),
                                                   count_Transaction_Amount=df.Transaction_Amount.count(),
                                                   max_Date_transaction=df.Date_transaction.max()
                                                  )
df_eng_feat_cust = df_eng_feat_cust.assign(nb_days_since_last_transactions = literal_column('INTERVAL(PERIOD(max_Date_transaction, CURRENT_DATE)) DAY(4)',type_= INTEGER))
df_eng_feat_cust = df_eng_feat_cust[['CustomerID','total_Transaction_Amount','avg_Transaction_Amount','count_Transaction_Amount','nb_days_since_last_transactions']]
df_eng_feat_cust

<p style = 'font-size:16px;font-family:Arial;'><b>Spending Category Distribution</b></p>

In [ ]:
df_eng_feat_cat = df.groupby('Category').agg({'Transaction_Amount':['sum','mean','std','min','max','median']})
df_eng_feat_cat = df_eng_feat_cat.join(
    df[['Category','Transaction_Amount']].groupby("Category").percentile(0.25),
    on = 'Category',
    how = 'inner',
    rprefix = 'r'
)[df_eng_feat_cat.columns + ['percentile_Transaction_Amount']]
df_eng_feat_cat = df_eng_feat_cat.assign(quartile_1_Transaction_Amount=df_eng_feat_cat.percentile_Transaction_Amount)
df_eng_feat_cat = df_eng_feat_cat[[c for c in df_eng_feat_cat.columns if c not in ['percentile_Transaction_Amount']]]
df_eng_feat_cat = df_eng_feat_cat.join(
    df[['Category','Transaction_Amount']].groupby("Category").percentile(0.25),
    on = 'Category',
    how = 'inner',
    rprefix = 'r'
)[df_eng_feat_cat.columns + ['percentile_Transaction_Amount']]
df_eng_feat_cat = df_eng_feat_cat.assign(quartile_3_Transaction_Amount=df_eng_feat_cat.percentile_Transaction_Amount)
df_eng_feat_cat = df_eng_feat_cat[[c for c in df_eng_feat_cat.columns if c not in ['percentile_Transaction_Amount']]]
df_eng_feat_cat = df_eng_feat_cat.assign(var_feature = 'test')
df_eng_feat_cat

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>4. Store the data transformations</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We are storing the transformation here. So, even if underlying data varies, the data transformation steps remain same.</p>

In [ ]:
df_eng_feat_cust = df_eng_feat_cust.create_view('FEAT_ENG_CUST')
df_eng_feat_cust

In [ ]:
df_eng_feat_cat = df_eng_feat_cat.create_view('FEAT_ENG_CAT')
df_eng_feat_cat

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>5. Ingest the features from <code>df_eng_feat_cust</code> datasource</b>
<p style = 'font-size:18px;font-family:Arial;'><b>5.1 See the mind_map for Feature Store</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Before ingesting the features, first check the mind map of feature store.
Since no features are ingested and no datasets are built, it should be empty.</p>

In [ ]:
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>5.2 Ingest the features</b></p>

In [ ]:
# Ingest the features for data source `df_eng_feat_cust`.
fp = fs.get_feature_process(object=df_eng_feat_cust,
                            entity='CustomerID',
                            features=[
                                'total_Transaction_Amount', 
                                'avg_Transaction_Amount', 
                                'count_Transaction_Amount', 
                                'nb_days_since_last_transactions'],
                            description='Feature Process for Customers'
                           )
# Run it to ingest the features.
fp.run()

<p style = 'font-size:18px;font-family:Arial;'><b>5.3 See the mind_map for Feature Store</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We ingested three features—<code>total_Transaction_Amount</code>, <code>avg_Transaction_Amount</code>, <code>count_Transaction_Amount</code> and <code>nb_days_since_last_transactions</code>—from a single feature process. This demonstrates how multiple related features can be managed and tracked together within the feature store, maintaining their lineage to the originating process.</p>

In [ ]:
# Features are ingested. Before we move to other APIs, look at mind map.
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>5.4 Explore FeatureStore for CustomerID entity</b></p>
<p style = 'font-size:16px;font-family:Arial;'><b>list feature_processes</b></p>

In [ ]:
# List the feature processes.
fs.list_feature_processes()

<p style = 'font-size:16px;font-family:Arial;'><b>list feature_catalogs</b></p>

In [ ]:
# Look at feature catalogs.
fs.list_feature_catalogs()

<p style = 'font-size:16px;font-family:Arial;'><b>list feature versions</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Get the feature catalog</p>

In [ ]:
fc = fs.get_feature_catalog()

<p style = 'font-size:16px;font-family:Arial;'>list feature versions</p>

In [ ]:
# Get the feature catalog and examine the versions.
fc.list_feature_versions()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>6. Ingest the features from <code>df_eng_feat_cat</code> datasource</b>
<p style = 'font-size:18px;font-family:Arial;'><b>6.1 Ingest the features</b></p>

In [ ]:
# Let's ingest features from another data source `df_eng_feat_cat`.
fp2 = fs.get_feature_process(object=df_eng_feat_cat,
                            entity='Category',
                            features=['sum_Transaction_Amount',
                                      'mean_Transaction_Amount',
                                      'std_Transaction_Amount',
                                      'min_Transaction_Amount',
                                      'max_Transaction_Amount',
                                      'median_Transaction_Amount',
                                      'quartile_1_Transaction_Amount',
                                      'quartile_3_Transaction_Amount',
                                      'var_feature'],
                            description='Feature Process for Category'
                           )
fp2.run()

<p style = 'font-size:18px;font-family:Arial;'><b>6.2 See the mind_map for Feature Store</b></p>

In [ ]:
# Let's look at mind map now.
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>6.3 Explore the FeatureStore for the operations that have been done so far</b></p>
<p style = 'font-size:16px;font-family:Arial;'><b>List feature processes</b></p>

In [ ]:
# # List the feature processes.
fs.list_feature_processes()

<p style = 'font-size:16px;font-family:Arial;'><b>List feature catalogs</b></p>

In [ ]:
# Look at feature catalogs.
display.max_rows=20
fs.list_feature_catalogs()

<p style = 'font-size:16px;font-family:Arial;'><b>List feature versions</b></p>

In [ ]:
# Get the feature catalog and examine the versions.
fc.list_feature_versions()

<p style = 'font-size:16px;font-family:Arial;'><b>List dataset catalogs</b></p>

In [ ]:
# Let's look at dataset catalogs. Since no dataset is created, it should be empty.
fs.list_dataset_catalogs()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>7. Build Dataset</b>
<p style = 'font-size:18px;font-family:Arial;'><b>7.1 Get the dataset catalog</b></p>

In [ ]:
# Let's build dataset in Dataset catalog. First get the pointer for Dataset catalog.
dc = fs.get_dataset_catalog()

<p style = 'font-size:18px;font-family:Arial;'><b>7.2 Build the dataset for <code>df_eng_feat_cust</code></b></p>

In [ ]:
df = dc.build_dataset(entity='CustomerID',
                      selected_features={'avg_Transaction_Amount': fp.process_id,
                                         'total_Transaction_Amount': fp.process_id},
                      view_name='CustID_Transactions_1'
                     )
df

<p style = 'font-size:16px;font-family:Arial;'>Verify the data</p>

In [ ]:
df_eng_feat_cust[df.CustomerID.isin([467220, 684935, 642109])]

<p style = 'font-size:18px;font-family:Arial;'><b>7.3 See the mind_map for Feature Store</b></p>

In [ ]:
# Let's look at mind map.
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>7.4 Build the time series dataset for <code>df_eng_feat_cust</code></b></p>

In [ ]:
# Let's build a time series. 
df = dc.build_time_series(entity='Category',
                          selected_features={'std_Transaction_Amount': fp2.process_id,
                                             'max_Transaction_Amount': fp2.process_id,
                                             'median_Transaction_Amount': fp2.process_id,
                                             'quartile_3_Transaction_Amount': fp2.process_id},
                          view_name='Cat_Transactions_1'
                         )
df

<p style = 'font-size:18px;font-family:Arial;'><b>7.4 See the mind_map for Feature Store</b></p>

In [ ]:
# Let's look at mind map.
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>7.5 List dataset catalogs </b></p>

In [ ]:
# Let's look at Dataset catalog. Two datasets created. Hence both should be available.
fs.list_dataset_catalogs()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>8. Feature Ingestion with filters</b>
<p style = 'font-size:18px;font-family:Arial;'><b>8.1 Load the sales data</b></p>

In [ ]:
# Ingest the features with filters.
load_example_data('dataframe', 'sales')
df = DataFrame("sales")
df

<p style = 'font-size:18px;font-family:Arial;'><b>8.2 Ingest the features</b></p>

In [ ]:
# Ingest the features only when entities has 'Inc' in it.
fp3 = fs.get_feature_process(object=df, 
                             entity='accounts', 
                             features=['Jan', 'Feb'])
fp3.run(filters=df.accounts.str.contains('Inc')==1)

<p style = 'font-size:18px;font-family:Arial;'><b>8.3 See the mind_map for Feature Store</b></p>

In [ ]:
# Let's look at mind map to see features are ingested or not. 
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>8.4 Look at the feature table</b></p>

In [ ]:
# Let's verify whether features ingested for that features or not. Look at catalog table.
fc.list_feature_versions()

In [ ]:
DataFrame(in_schema(fs.repo, "FS_T_4fb46368_0c60_21bf_20a1_b0a3948fedde"))

<p style = 'font-size:18px;font-family:Arial;'><b>8.5 Ingest the features for an incremental load</b></p>

<p style = 'font-size:16px;font-family:Arial;'><b>let's first update the values of features. Otherwise, update is not visible</b></p>

In [ ]:
execute_sql('update sales set Feb=Feb*2')

<p style = 'font-size:16px;font-family:Arial;'><b>Store the values for increamental load</b></p>

In [ ]:
fp4 = fs.get_feature_process(object=df, 
                             entity='accounts', 
                             features=['Jan', 'Feb']
                            )
fp4.run()

<p style = 'font-size:18px;font-family:Arial;'><b>8.6 Build the dataset</b></p>

In [ ]:
# Let's first look at the data. For that, let's build a dataset.
df_sales_jan_feb = dc.build_dataset(entity='accounts',
                                    selected_features={
                                        'Jan': fp4.process_id,
                                        'Feb': fp4.process_id
                                    },
                                   view_name='sales_jan_feb')

In [ ]:
df_sales_jan_feb.sort('accounts')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>9. Ingest the features at specific time</b>
<p style = 'font-size:18px;font-family:Arial;'><b>9.1 Create new data domain for specific time ingestion</b></p>

In [ ]:
# For that, let's create another data domain.
fs_sales_analytics = FeatureStore(repo='enterprise_feature_repo', data_domain='Sales_Analytics')

<p style = 'font-size:18px;font-family:Arial;'><b>9.2 See the FeatureStore for 'Sales_Analytics' data domain</b></p>

In [ ]:
# Let's first see what this data domain has. It should be empty.
fs_sales_analytics.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>9.3 Drop and load the sales data</b></p>

In [ ]:
# Let's drop the sales table and load it again.
db_drop_table('sales')
load_example_data('dataframe', 'sales')
df = DataFrame("sales")

<p style = 'font-size:18px;font-family:Arial;'><b>9.4 Ingest the features</b></p>

In [ ]:
# Let's first ingest the features.
fp5 = fs_sales_analytics.get_feature_process(object=df, 
                                             entity='accounts', 
                                             features=['Jan', 'Feb', 'Mar', 'Apr']
                                             )
fp5.run()

In [ ]:
fs_sales_analytics.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>9.5 Look at Jan's values first</b></p>

In [ ]:
fc_sales_analytics = fs_sales_analytics.get_feature_catalog()
fc_sales_analytics.list_feature_versions()

In [ ]:
# Let's look at Jan's values first.
sales_jan = DataFrame(in_schema(fs.repo, "FS_T_feb531d5_375e_5831_05d7_3ac2ac2a7965")).as_of(valid_time='current')
sales_jan[sales_jan.feature_id == 16]

<p style = 'font-size:18px;font-family:Arial;'><b>9.5 Update the feature values to simulate the data change</b></p>

In [ ]:
execute_sql('update sales set Jan=Jan*2, Feb=Feb*2, Mar=Mar*2, Apr=Apr*2')

<p style = 'font-size:18px;font-family:Arial;'><b>9.6 Ingest the same features again</b></p>

In [ ]:
fp6 = fs_sales_analytics.get_feature_process(object=df, 
                                             entity='accounts', 
                                             features=['Jan', 'Feb', 'Mar', 'Apr']
                                             )
fp6.run()

In [ ]:
# Let's look at mind map. No new feature process initiated. Hence it should be same as previous state.
fs_sales_analytics.mind_map()

<p style = 'font-size:16px;font-family:Arial;'><b>look at features again. Observe that only the new values appeared</b></p>
<p style = 'font-size:16px;font-family:Arial;'><b>Note:</b> Old values are not visible since function as_of selected only current records.</p>

In [ ]:
sales_jan = DataFrame(in_schema(fs.repo, "FS_T_feb531d5_375e_5831_05d7_3ac2ac2a7965")).as_of(valid_time='current')
sales_jan[sales_jan.feature_id == 16]

<p style = 'font-size:16px;font-family:Arial;'><b>Select all values. Then notice the difference</b></p>

In [ ]:
sales_jan = DataFrame(in_schema(fs.repo, "FS_T_feb531d5_375e_5831_05d7_3ac2ac2a7965")).as_of(valid_time=None)
sales_jan[sales_jan.feature_id == 16].sort(['accounts', 'valid_start'])

<p style = 'font-size:18px;font-family:Arial;'><b>9.7 Update the data again to ingest for a specific time</b></p>

In [ ]:
# Double the data again. This time ingest for a specific time.
# Update the feature values to simulate the data change. 
execute_sql('update sales set Jan=Jan*2, Feb=Feb*2, Mar=Mar*2, Apr=Apr*2')

<p style = 'font-size:18px;font-family:Arial;'><b>9.8 Ingest feature value for specific time</b></p>

In [ ]:
# Let's first ingest the features.
fp7 = fs_sales_analytics.get_feature_process(object=df, 
                                             entity='accounts', 
                                             features=['Jan', 'Feb', 'Mar', 'Apr'],
                                             )
fp7.run(as_of='2025-09-08 04:51:59.000000+00:00')

In [ ]:
# Let's verify the data again.
sales_jan = DataFrame(in_schema(fs.repo, "FS_T_feb531d5_375e_5831_05d7_3ac2ac2a7965")).as_of(valid_time=None)
sales_jan[sales_jan.feature_id == 16].sort(['accounts', 'valid_start'])

<p style = 'font-size:18px;font-family:Arial;'><b>9.9 See the mind_map for fs_sales_analytics FeatureStore</b></p>

In [ ]:
# Remove the whole data domain.
# Before removing it, Let's first look at feature store for this data domain.
fs_sales_analytics.mind_map()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>10. Remove 'Sales_Analytics' DataDomain</b>
<p style = 'font-size:18px;font-family:Arial;'><b>10.1 Remove DataDomain</b></p>

In [ ]:
# Remove it and then again look at mind map.
fs_sales_analytics.remove_data_domain()

<p style = 'font-size:18px;font-family:Arial;'><b>10.2 See the FeatureStore for Sales_analytics data domain</b></p>

In [ ]:
# Verify the objects.
fs_sales_analytics.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>10.3 See the FeatureStore Analytics data domain</b></p>

In [ ]:
# Also verify the objects of fs. Since only data domain sales analytics is removed, fs objects should remain as it is.
fs.mind_map()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>11. Dataset Management</b>
<p style = 'font-size:18px;font-family:Arial;'><b>11.1 List datasets</b></p>

In [ ]:
dc.list_datasets()

<p style = 'font-size:18px;font-family:Arial;'><b>11.2 List features</b></p>

In [ ]:
dc.list_features()

<p style = 'font-size:18px;font-family:Arial;'><b>11.3 List Entities</b></p>

In [ ]:
dc.list_entities()

<p style = 'font-size:18px;font-family:Arial;'><b>11.4 Archive datasets</b></p>

<p style = 'font-size:16px;font-family:Arial;'><b>Get the dataset</b></p>


In [ ]:
#Replace the dataset_id from the list entities step - name:sales_jan_feb
dc.archive_datasets('3fab75f7-30f2-4faa-b32a-17e703c6258a')

In [ ]:
dc.list_datasets()

<p style = 'font-size:18px;font-family:Arial;'><b>11.5 Delete datasets</b></p>

In [ ]:
#Replace the dataset_id from the list entities step - name:sales_jan_feb
dc.delete_datasets('3fab75f7-30f2-4faa-b32a-17e703c6258a')

In [ ]:
# Verify the datasets after deleting dataset
dc.list_datasets()

<p style = 'font-size:18px;font-family:Arial;'><b>11.6 List features</b></p>

In [ ]:
# Verify the features after deleting dataset
dc.list_features()

<p style = 'font-size:18px;font-family:Arial;'><b>11.7 List entities</b></p>

In [ ]:
# Verify the Entities after deleting dataset
dc.list_entities()

<p style = 'font-size:18px;font-family:Arial;'><b>11.8 See mind_map for FeatureStore after deleting dataset</b></p>

In [ ]:
# Let's look at mind map of Feature store now. Deleted dataset should not appear.
fs.mind_map()

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>12. Dataset</b>
<p style = 'font-size:18px;font-family:Arial;'><b>12.1 Explore Dataset methods</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Get the dataset </p>

In [ ]:
#Replace the dataset_id from the list datasets step - name:CustID_Transactions_1
ds = dc.get_dataset('a810fc39-c2e1-4d87-95d5-ae623751bcc4')

<p style = 'font-size:18px;font-family:Arial;'><b>12.2 Explore properties</b></p>
<p style = 'font-size:16px;font-family:Arial;'><b>features:</b> </p>

In [ ]:
ds.features

<p style = 'font-size:16px;font-family:Arial;'><b>entity:</b> </p>

In [ ]:
ds.entity

<p style = 'font-size:16px;font-family:Arial;'><b>id:</b> </p>

In [ ]:
ds.id

<p style = 'font-size:16px;font-family:Arial;'><b>view_name:</b> </p>

In [ ]:
ds.view_name

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>13. Feature Management</b>
<p style = 'font-size:18px;font-family:Arial;'><b>13.1 List features</b></p>

In [ ]:
fc.list_features()

<p style = 'font-size:18px;font-family:Arial;'><b>13.2 list feature versions</b></p>

In [ ]:
fc.list_feature_versions()

In [ ]:
# Looks to be a bug. Investigation is in progress.
fc.features

In [ ]:
fc.entities

<p style = 'font-size:18px;font-family:Arial;'><b>13.3 Archive features</b></p>

In [ ]:
# Let's archive a feature which is associated with any dataset. 
fc.archive_features('Jan')

In [ ]:
# Observe that the Jan record is closed.
fc.list_features()

<p style = 'font-size:18px;font-family:Arial;'><b>13.4 Look at the feature values also. Even they all should be closed</b></p>

In [ ]:
# Look at the feature values also. Even they all should be closed.
display.max_rows=30
fc.list_feature_versions()

In [ ]:
ndf = DataFrame(in_schema(fs.repo, "FS_T_4fb46368_0c60_21bf_20a1_b0a3948fedde")).as_of(valid_time='current')
ndf[ndf.feature_id==14]

In [ ]:
# Retrieve all the records. Since Jan is still not deleted, as_of with valid_time as None should still show results.
# However, all the records should be closed.
ndf = DataFrame(in_schema(fs.repo, "FS_T_4fb46368_0c60_21bf_20a1_b0a3948fedde")).as_of(valid_time=None)
ndf[ndf.feature_id==14]

<p style = 'font-size:18px;font-family:Arial;'><b>13.5 Deletes features</b></p>

In [ ]:
# Let's delete the archived feature.
fc.delete_features('Jan')

In [ ]:
# Verify Jan is completly removed or not.
fc.list_features()

<p style = 'font-size:18px;font-family:Arial;'><b>13.6 Let's look at the mind_map now</b></p>

In [ ]:
# Let's look at mind_map now. 
fs.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>13.7 List feature processes</b></p>

In [ ]:
# Let's remove the feature process. First look at feature processes.
fs.list_feature_processes()

<p style = 'font-size:18px;font-family:Arial;'><b>13.8 Achive feature process</b></p>

In [ ]:
#process_id where entity_id:accounts
fs.archive_feature_process('7449fefa-a33f-11f0-abbb-0242ac110003')

In [ ]:
# Let's look at feature processes. Observe that feature process '85dc42e8-670f-11f0-bb98-c934b24a960f' is also archived. It is closed.
fs.list_feature_processes()

In [ ]:
# Let's look at catalog data. Observe that feature Feb is also archived. It is closed.
fc.list_features()

<p style = 'font-size:18px;font-family:Arial;'><b>13.9 Delete feature process</b></p>

In [ ]:
#process_id where entity_id:accounts
fs.delete_feature_process('7449fefa-a33f-11f0-abbb-0242ac110003')

<p style = 'font-size:18px;font-family:Arial;'><b>13.10 Let's look at mind map now</b></p>

In [ ]:
fs.mind_map()

In [ ]:
# look at mind map now. All should be empty.
fs_sales_analytics.mind_map()

<p style = 'font-size:18px;font-family:Arial;'><b>13.11 Remove data domain</b></p>

In [ ]:
# Let's remove the whole data domain. That should remove all objects.
fs.remove_data_domain()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>14. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;'> <b>Work Tables and Views </b></p>

In [ ]:
db_drop_view('FEAT_ENG_CUST')

In [ ]:
db_drop_view('FEAT_ENG_CAT')

In [ ]:
db_drop_table('transactions')

In [ ]:
db_drop_table('sales')

In [ ]:
remove_context()

<p style = 'font-size:18px;font-family:Arial;'><b>14.1 Delete the Feature Store</b></p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

<p style = 'font-size:16px;font-family:Arial;'><b>Note :</b> This will drop the database if all objects are removed.</p>

In [ ]:
fs.delete()

In [ ]:
# Let's verify the repos. It should be empty.
FeatureStore.list_repos()

In [ ]:
# Finally disconnect the DB Admin from Vantage.
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>